# 实验报告
<font size=4>
    
+ **姓名：于成俊**
+ **学号：2112066**
+ **专业：密码科学与技术**

</font>

## 实验要求
    
<font size=4>
    

+ 基本要求：
    + a) 基于 Watermelon-train1数据集（只有离散属性），构造ID3决策树；
    + b) 基于构造的 ID3 决策树，对数据集 Watermelon-test1进行预测，输出分类精度；
+ 中级要求：
    + a) 对数据集Watermelon-train2，构造C4.5或者CART决策树，要求可以处理连续型属性；
    + b) 对测试集Watermelon-test2进行预测，输出分类精度；
+ 高级要求：
    + 使用任意的剪枝算法对构造的决策树（基本要求和中级要求构造的树）进行剪枝，观察测试集合的分类精度是否有提升，给出分析过程。
</font>


## 实验流程

首先导入相关包

In [4]:
import csv
import math

编写读取数据集函数

In [5]:
# 读取数据集
def read_dataset(filename):
    # 由于文件是‘GBK’编码，所以采用GBK的方式打开文件
    with open(filename, 'r', encoding='GBK') as f:
        reader = csv.reader(f)
        return list(reader)

### 基本要求
#### 构造ID3决策树：

**构造ID3决策树类:**

In [6]:
class ID3:
    def __init__(self):
        self.root = None

    class Node:
        def __init__(self, attribute=None, label=None):
            self.attribute = attribute
            self.label = label
            self.children = {}

    def fit(self, data, labels):
        # 使用 ID3 算法构建决策树，得到根节点
        self.root = self._id3(data, labels, list(range(len(data[0]))))

    def _id3(self, data, labels, attributes):
        #  将labels 转换为集合（set）,从而计算不同标签的数量
        if len(set(labels)) == 1:
            # 如果只有一个标签，说明所有数据点都属于同一标签，则创建一个该标签的叶子节点
            return self.Node(label=labels[0])
        # 如果没有可用的属性（特征），则同样创建一个叶子节点，节点的标签为出现次数最多的标签。
        if len(attributes) == 0:
            return self.Node(label=max(set(labels), key=labels.count))
        # 选择具有最大信息增益的属性
        best_attr = max(attributes, key=lambda attr: self._gain(data, labels, attr))
        # 创建一个具有最大信息增益的属性的节点
        node = self.Node(attribute=best_attr)
        # 遍历best_attr的所有可能取值。
        for value in set(row[best_attr] for row in data):
            # 根据best_attr的取值将数据和标签进行划分，创建子集。
            child_data = [row for row, l in zip(data, labels) if row[best_attr] == value]
            child_labels = [l for row, l in zip(data, labels) if row[best_attr] == value]
            # 如果子集为空，说明所有数据都具有相同的属性值，创建一个叶子节点，节点的标签为目标变量中出现次数最多的标签
            if not child_data:
                # 将子节点添加到当前节点的子节点中。
                node.children[value] = self.Node(label=max(set(labels), key=labels.count))
            else:
                # 递归调用 _id3 方法，继续构建决策树。
                node.children[value] = self._id3(child_data, child_labels, [attr for attr in attributes if attr != best_attr])

        return node

    # 计算信息增益
    def _gain(self, data, labels, attr):
        # 计算整体数据集的熵
        entropy = self._entropy(labels)
        # 获取属性列的值
        values = [row[attr] for row in data]
        # 将数据集按照属性值分割成不同的分区
        partitions = [[t for v, t in zip(values, labels) if v == value] for value in set(values)]
        # 计算信息增益
        return entropy - sum(len(part) / len(labels) * self._entropy(part) for part in partitions)

    def _entropy(self, labels):
        # 计算每个标签在标签集中的出现次数
        counts = [labels.count(value) for value in set(labels)]
        # 计算信息熵
        return -sum(count / len(labels) * math.log2(count / len(labels)) for count in counts)

    def predict(self, data):
        # 返回一个列表，包含对每个数据点的预测结果
        return [self._predict(row, self.root) for row in data]

    def _predict(self, row, node):
        # 如果节点是叶子节点，直接返回叶子节点的标签值
        if node.label is not None:
            return node.label
        # 如果节点不是叶子节点，继续递归预测
        return self._predict(row, node.children[row[node.attribute]])

**使用ID3决策树进行训练和预测：**

In [7]:
# 使用ID3决策树进行训练和预测
def ID3_train_predict():
    # 读取训练集和测试集数据
    train_data = read_dataset('Watermelon-train1.csv')
    test_data = read_dataset('Watermelon-test1.csv')

    # 创建 ID3 决策树实例
    id3 = ID3()

    # 使用训练集训练 ID3 决策树
    id3.fit([row[1:-1] for row in train_data], [row[-1] for row in train_data])

    # 对测试集进行预测
    predictions = id3.predict([row[1:-1] for row in test_data])

    # 计算分类精度
    accuracy = sum(p == t[-1] for p, t in zip(predictions, test_data)) / len(test_data)

    # 打印分类精度
    print(f'分类精度: {accuracy}')

In [8]:
ID3_train_predict()

分类精度: 0.7272727272727273


### 中级要求
#### 构造CART决策树：

**构造CART决策树类:**

In [9]:
class CART:
    def __init__(self):
        self.root = None

    class Node:
        def __init__(self, attribute=None, threshold=None, label=None, left=None, right=None):
            self.attribute = attribute
            self.threshold = threshold
            self.label = label
            self.left = left
            self.right = right

    def fit(self, data, labels):
        # 使用 CART 决策树算法构建决策树，将根节点保存在 self.root 中
        self.root = self._cart(data, labels, list(range(len(data[0]))))

    def _cart(self, data, labels, attributes):
        # 如果标签集唯一，创建叶子节点并返回
        if len(set(labels)) == 1:
            return self.Node(label=labels[0])

        # 如果没有可用的属性，创建叶子节点并返回，标签为目标变量中最频繁的值
        if len(attributes) == 0:
            return self.Node(label=max(set(labels), key=labels.count))

        # 寻找最佳分裂属性和阈值
        best_attr, best_threshold = self._best_split(data, labels, attributes)
        node = self.Node(attribute=best_attr, threshold=best_threshold)

        # 根据最佳分裂属性和阈值分割数据集
        left_data, left_labels, right_data, right_labels = self._split_data(data, labels, best_attr, best_threshold)

        # 如果左子集为空，创建叶子节点并返回，标签为左子集中最频繁的值
        if not left_data:
            node.label = max(set(labels), key=labels.count)
        else:
            # 递归构建左子树和右子树
            node.left = self._cart(left_data, left_labels, [attr for attr in attributes if attr != best_attr])
            node.right = self._cart(right_data, right_labels, [attr for attr in attributes if attr != best_attr])

        return node

    def _best_split(self, data, labels, attributes):
        # 初始化最佳分裂属性、最佳阈值和最小的基尼指数
        best_attr, best_threshold, min_gini = None, None, float('inf')

        # 遍历所有属性
        for attr in attributes:
            # 获取当前属性的所有取值
            values = [row[attr] for row in data]

            # 遍历当前属性的每个唯一取值，将数据集分割为左右两部分
            for value in set(values):
                left_labels = [t for v, t in zip(values, labels) if v < value]
                right_labels = [t for v, t in zip(values, labels) if v >= value]

                # 计算基尼指数
                gini = self._gini(left_labels) * len(left_labels) / len(labels) + self._gini(right_labels) * len(
                    right_labels) / len(labels)

                # 更新最小基尼指数及相应的属性和阈值
                if gini < min_gini:
                    best_attr, best_threshold, min_gini = attr, value, gini

        # 返回最佳分裂属性和阈值
        return best_attr, best_threshold

    def _split_data(self, data, labels, attr, threshold):
        # 初始化左右两部分的数据和标签集
        left_data, left_labels, right_data, right_labels = [], [], [], []

        # 遍历数据集中的每一行及其对应的目标变量
        for row, l in zip(data, labels):
            # 根据指定的属性和阈值进行分割
            if row[attr] < threshold:
                # 如果当前样本的指定属性值小于阈值，加入左侧部分
                left_data.append(row)
                left_labels.append(l)
            else:
                # 如果当前样本的指定属性值大于等于阈值，加入右侧部分
                right_data.append(row)
                right_labels.append(l)

        # 返回左右两部分的数据和标签集
        return left_data, left_labels, right_data, right_labels

    def _gini(self, labels):
        # 初始化基尼指数
        gini = 1 - sum((labels.count(value) / len(labels)) ** 2 for value in set(labels))
        return gini

    def predict(self, data):
        # 返回一个列表，其中每个元素是对应样本的预测值
        return [self._predict(row, self.root) for row in data]

    def _predict(self, row, node):
        # 如果当前节点是叶子节点，直接返回节点的标签作为预测值
        if node.label is not None:
            return node.label
        # 如果当前节点不是叶子节点，根据节点的划分条件继续向下遍历
        if row[node.attribute] < node.threshold:
            # 如果样本的指定属性值小于节点的阈值，则递归预测左子树
            return self._predict(row, node.left)
        # 如果样本的指定属性值大于等于节点的阈值，则递归预测右子树
        return self._predict(row, node.right)

**使用CART决策树进行训练和预测：**

In [10]:
def CRAT_train_predict():
    # 读取训练集和测试集数据
    train_data = read_dataset('Watermelon-train2.csv')
    test_data = read_dataset('Watermelon-test2.csv')

    # 创建 CART 决策树实例
    cart = CART()

    # 使用训练集训练 CART 决策树
    cart.fit([row[1:-1] for row in train_data], [row[-1] for row in train_data])

    # 对测试集进行预测
    predictions = cart.predict([row[1:-1] for row in test_data])

    # 计算分类精度
    accuracy = sum(p == t[-1] for p, t in zip(predictions, test_data)) / len(test_data)

    # 打印分类精度
    print(f'分类精度: {accuracy}')

In [11]:
CRAT_train_predict()

分类精度: 0.6666666666666666


### 高级要求

首先对ID3树进行剪枝，剪枝的策略为如果所有子节点都是叶子节点且它们的标签相同，就将当前节点替换为叶子节点。这个剪枝算法在决策树构建完毕后进行，以提高模型的泛化能力。这种策略的核心思想是当一个节点的所有子节点都是叶子节点且它们的标签相同时，可以认为这个节点对训练数据进行了过度拟合。因此，
将这个节点替换为一个叶子节点。其中，剪枝的ID3类比普通的ID3类多了一个prune函数，并且在fit函数的最后调用这个函数，其他函数与普通的ID3类完全相同。为了方便判断一个节点是否是叶子节点，还在Node类中加了一个判断函数。

**剪枝的ID3树如下：**

In [12]:
class ID3_Pruning:
    def __init__(self):
        self.root = None

    class Node:
        def __init__(self, attribute=None, label=None):
            self.attribute = attribute
            self.label = label
            self.children = {}
        # 判断节点是否为叶子节点
        def is_leaf(self):
            return not bool(self.children)

    def prune(self, node):
        # 如果所有的子节点都是叶子节点
        if all(child.is_leaf() for child in node.children.values()):
            # 获取所有子节点的标签
            labels = [child.label for child in node.children.values()]
            # 如果所有子节点的标签都相同，将当前节点替换为叶子节点
            if len(set(labels)) == 1:
                return self.Node(label=labels[0])
        # 递归地对每个子节点进行剪枝
        for child in node.children.values():
            self.prune(child)
        # 返回当前节点
        return node

    def fit(self, data, labels):
        # 使用 ID3 算法构建决策树，得到根节点
        self.root = self._id3(data, labels, list(range(len(data[0]))))
        self.root = self.prune(self.root)  # 在训练结束后进行剪枝

    def _id3(self, data, labels, attributes):
        #  将labels 转换为集合（set）,从而计算不同标签的数量
        if len(set(labels)) == 1:
            # 如果只有一个标签，说明所有数据点都属于同一标签，则创建一个该标签的叶子节点
            return self.Node(label=labels[0])
        # 如果没有可用的属性（特征），则同样创建一个叶子节点，节点的标签为出现次数最多的标签。
        if len(attributes) == 0:
            return self.Node(label=max(set(labels), key=labels.count))
        # 选择具有最大信息增益的属性
        best_attr = max(attributes, key=lambda attr: self._gain(data, labels, attr))
        # 创建一个具有最大信息增益的属性的节点
        node = self.Node(attribute=best_attr)
        # 遍历best_attr的所有可能取值。
        for value in set(row[best_attr] for row in data):
            # 根据best_attr的取值将数据和标签进行划分，创建子集。
            child_data = [row for row, l in zip(data, labels) if row[best_attr] == value]
            child_labels = [l for row, l in zip(data, labels) if row[best_attr] == value]
            # 如果子集为空，说明所有数据都具有相同的属性值，创建一个叶子节点，节点的标签为目标变量中出现次数最多的标签
            if not child_data:
                # 将子节点添加到当前节点的子节点中。
                node.children[value] = self.Node(label=max(set(labels), key=labels.count))
            else:
                # 递归调用 _id3 方法，继续构建决策树。
                node.children[value] = self._id3(child_data, child_labels,
                                                 [attr for attr in attributes if attr != best_attr])

        return node

    def _gain(self, data, labels, attr):
        # 计算整体数据集的熵
        entropy = self._entropy(labels)
        # 获取属性列的值
        values = [row[attr] for row in data]
        # 将数据集按照属性值分割成不同的分区
        partitions = [[l for v, l in zip(values, labels) if v == value] for value in set(values)]
        # 计算信息增益
        return entropy - sum(len(part) / len(labels) * self._entropy(part) for part in partitions)

    def _entropy(self, labels):
        # 计算每个标签在标签集中的出现次数
        counts = [labels.count(value) for value in set(labels)]
        # 计算信息熵
        return -sum(count / len(labels) * math.log2(count / len(labels)) for count in counts)

    def predict(self, data):
        # 返回一个列表，包含对每个数据点的预测结果
        return [self._predict(row, self.root) for row in data]

    def _predict(self, row, node):
        # 如果节点是叶子节点，直接返回叶子节点的标签值
        if node.label is not None:
            return node.label
        # 如果节点不是叶子节点，继续递归预测
        return self._predict(row, node.children[row[node.attribute]])

**使用可以剪枝的ID3决策树进行训练和预测：**

In [13]:
# 使用可以剪枝的ID3决策树进行训练和预测
def ID3_Pruning_train_predict():
    # 读取训练集和测试集数据
    train_data = read_dataset('Watermelon-train1.csv')
    test_data = read_dataset('Watermelon-test1.csv')

    # 创建 ID3 决策树实例
    id3_Pruning = ID3_Pruning()

    # 使用训练集训练 ID3 决策树
    id3_Pruning.fit([row[1:-1] for row in train_data], [row[-1] for row in train_data])

    # 对测试集进行预测
    predictions = id3_Pruning.predict([row[1:-1] for row in test_data])

    # 计算分类精度
    accuracy = sum(p == t[-1] for p, t in zip(predictions, test_data)) / len(test_data)

    # 打印分类精度
    print(f'分类精度: {accuracy}')

In [14]:
ID3_Pruning_train_predict()

分类精度: 0.7272727272727273


对CART树的剪枝策略与ID3树相同，且也是多了一个prune函数，并在fit函数最后调用prune函数。在Node类中，也有一个判断是否为叶子节点的函数。

**剪枝的CART树如下：**

In [15]:
class CART_Pruning:
    def __init__(self):
        self.root = None

    class Node:
        def __init__(self, attribute=None, threshold=None, label=None, left=None, right=None):
            self.attribute = attribute
            self.threshold = threshold
            self.label = label
            self.left = left
            self.right = right

        # 判断节点是否为叶子节点
        def is_leaf(self):
            return self.left is None and self.right is None

    def prune(self, node):
        # 如果当前节点有左右两个子节点
        if node.left and node.right:
            # 如果左右子节点都是叶子节点且它们的标签相同
            if node.left.is_leaf() and node.right.is_leaf() and node.left.label == node.right.label:
                # 将当前节点标签设为左子节点的标签，且删除左右子节点
                node.label = node.left.label
                node.left = None
                node.right = None
        else:
            # 如果有左子节点，则递归对左子节点进行剪枝
            if node.left:
                self.prune(node.left)
            # 如果有右子节点，则递归对右子节点进行剪枝
            if node.right:
                self.prune(node.right)
        # 返回当前节点
        return node

    def fit(self, data, target):
        # 使用 CART 决策树算法构建决策树，将根节点保存在 self.root 中
        self.root = self._cart(data, target, list(range(len(data[0]))))
        self.root = self.prune(self.root)  # 在训练结束后进行剪枝

    def _cart(self, data, labels, attributes):
        # 如果标签集唯一，创建叶子节点并返回
        if len(set(labels)) == 1:
            return self.Node(label=labels[0])

        # 如果没有可用的属性，创建叶子节点并返回，标签为目标变量中最频繁的值
        if len(attributes) == 0:
            return self.Node(label=max(set(labels), key=labels.count))

        # 寻找最佳分裂属性和阈值
        best_attr, best_threshold = self._best_split(data, labels, attributes)
        node = self.Node(attribute=best_attr, threshold=best_threshold)

        # 根据最佳分裂属性和阈值分割数据集
        left_data, left_labels, right_data, right_labels = self._split_data(data, labels, best_attr, best_threshold)

        # 如果左子集为空，创建叶子节点并返回，标签为左子集中最频繁的值
        if not left_data:
            node.label = max(set(labels), key=labels.count)
        else:
            # 递归构建左子树和右子树
            node.left = self._cart(left_data, left_labels, [attr for attr in attributes if attr != best_attr])
            node.right = self._cart(right_data, right_labels, [attr for attr in attributes if attr != best_attr])

        return node

    def _best_split(self, data, labels, attributes):
        # 初始化最佳分裂属性、最佳阈值和最小的基尼指数
        best_attr, best_threshold, min_gini = None, None, float('inf')

        # 遍历所有属性
        for attr in attributes:
            # 获取当前属性的所有取值
            values = [row[attr] for row in data]

            # 遍历当前属性的每个唯一取值，将数据集分割为左右两部分
            for value in set(values):
                left_labels = [l for v, l in zip(values, labels) if v < value]
                right_labels = [l for v, l in zip(values, labels) if v >= value]

                # 计算基尼指数
                gini = self._gini(left_labels) * len(left_labels) / len(labels) + self._gini(right_labels) * len(
                    right_labels) / len(labels)

                # 更新最小基尼指数及相应的属性和阈值
                if gini < min_gini:
                    best_attr, best_threshold, min_gini = attr, value, gini

        # 返回最佳分裂属性和阈值
        return best_attr, best_threshold

    def _split_data(self, data, labels, attr, threshold):
        # 初始化左右两部分的数据和标签集
        left_data, left_labels, right_data, right_labels = [], [], [], []

        # 遍历数据集中的每一行及其对应的目标变量
        for row, l in zip(data, labels):
            # 根据指定的属性和阈值进行分割
            if row[attr] < threshold:
                # 如果当前样本的指定属性值小于阈值，加入左侧部分
                left_data.append(row)
                left_labels.append(l)
            else:
                # 如果当前样本的指定属性值大于等于阈值，加入右侧部分
                right_data.append(row)
                right_labels.append(l)

        # 返回左右两部分的数据和标签集
        return left_data, left_labels, right_data, right_labels

    def _gini(self, labels):
        # 初始化基尼指数
        gini = 1 - sum((labels.count(value) / len(labels)) ** 2 for value in set(labels))
        return gini

    def predict(self, data):
        # 返回一个列表，其中每个元素是对应样本的预测值
        return [self._predict(row, self.root) for row in data]

    def _predict(self, row, node):
        # 如果当前节点是叶子节点，直接返回节点的标签作为预测值
        if node.label is not None:
            return node.label
        # 如果当前节点不是叶子节点，根据节点的划分条件继续向下遍历
        if row[node.attribute] < node.threshold:
            # 如果样本的指定属性值小于节点的阈值，则递归预测左子树
            return self._predict(row, node.left)
        # 如果样本的指定属性值大于等于节点的阈值，则递归预测右子树
        return self._predict(row, node.right)

**使用可以剪枝的CART决策树进行训练和预测：**

In [16]:
# 使用可以剪枝的CART决策树进行训练和预测
def CRAT_Pruning_train_predict():
    # 读取训练集和测试集数据
    train_data = read_dataset('Watermelon-train2.csv')
    test_data = read_dataset('Watermelon-test2.csv')

    # 创建 CART 决策树实例
    cart_pruning = CART_Pruning()

    # 使用训练集训练 CART 决策树
    cart_pruning.fit([row[1:-1] for row in train_data], [row[-1] for row in train_data])

    # 对测试集进行预测
    predictions = cart_pruning.predict([row[1:-1] for row in test_data])

    # 计算分类精度
    accuracy = sum(p == t[-1] for p, t in zip(predictions, test_data)) / len(test_data)

    # 打印分类精度
    print(f'分类精度: {accuracy}')

In [17]:
CRAT_Pruning_train_predict()

分类精度: 0.6666666666666666


从结果可以看出，分类精度与没有使用剪枝是一样的。分析原因如下：
+   剪枝的目的是提高模型的泛化能力，减少过拟合。我认为，原本的决策树本身已经足够简单，剪枝操作并未剔除过拟合的部分，所以，并没有带来显著的性能改善。
+   本次实验的数据集的数据量比较小，模型难以在小数据集上学习复杂的关系。而过拟合通常发生在模型过于复杂，相反在数据集较小的情况下，模型可能会出现欠拟合的情况。而剪枝主要是针对过拟合的情况，所以精确度并未出现提升。